# Baseline (T)

In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import tensorflow as tf
import tensorflow_addons as tfa
from keras import layers

import librosa
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, OneOf

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import pickle
import argparse
import wandb
from wandb.keras import WandbCallback
wandb.init(project="DACON_235910", name="Baseline_T")

parser = argparse.ArgumentParser(description="Baseline_T")
parser.add_argument('--sampling_rate', default=16000, type=int)
parser.add_argument('--augmentation', default=True, type=bool)
parser.add_argument('--optimizer', default="sgd", type=str) # sgd or adam
parser.add_argument('--loss', default="fl", type=str) # bc or fl
parser.add_argument('--learning_rate', default=0.001, type=float)
parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--cv', default=10, type=int)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

sampling_rate = args.sampling_rate
augmentation = args.augmentation
optimizer = args.optimizer
loss = args.loss
learning_rate = args.learning_rate
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
cv = args.cv
seed = args.seed

def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
augment = Compose([
    OneOf([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
        TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
        Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
    ])
])

with open('data/train_df.pkl', 'rb') as f:
    train_df = pickle.load(f)
with open('data/test_df.pkl', 'rb') as f:
    test_df = pickle.load(f)
    
train_df.head()

wandb: Currently logged in as: gnoeyheat (use `wandb login --relogin` to force relogin)


,data,id,age,gender,respiratory_condition,fever_or_muscle_pain,covid19
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,24,female,0,1,0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2,51,male,0,0,0
2,"[2.7372453e-09, -1.0615647e-08, 5.2142607e-08,...",3,22,male,0,0,0
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4,29,female,1,0,0
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5,23,male,0,0,0


## Data Generation

In [2]:
# train_df = pd.read_csv("data/train_data.csv")
# test_df = pd.read_csv("data/test_data.csv")

# train_folder = "data/train/"
# test_folder = "data/test/"

# def dataset(folder, df):
#     dataset = []
#     for uid in tqdm(df['id']):
#         path = os.path.join(folder, str(uid).zfill(5)+'.wav')
#         y, sr = librosa.load(path, sr=sampling_rate)
#         y = librosa.util.normalize(y)
#         dataset.append([y])
#     dataset = pd.DataFrame(dataset, columns=['data'])
#     dataset = pd.concat([dataset, df], axis=1)
#     return dataset

# train_df = dataset(train_folder, train_df)
# test_df = dataset(test_folder, test_df)

# with open('train_df.pkl', 'wb') as f:
#     pickle.dump(train_df, f, pickle.HIGHEST_PROTOCOL)
# with open('test_df.pkl', 'wb') as f:
#     pickle.dump(test_df, f, pickle.HIGHEST_PROTOCOL)

## Preprocessing

In [3]:
train_df["len"] = train_df["data"].apply(lambda x : len(x))
test_df["len"] = test_df["data"].apply(lambda x : len(x))

train_df["len"].median(), test_df["len"].median()

(156480.0, 155520.0)

In [4]:
X = tf.keras.preprocessing.sequence.pad_sequences(train_df["data"],
                                                  maxlen=150000,
                                                  dtype="float32")[:, :, np.newaxis]
X_test = tf.keras.preprocessing.sequence.pad_sequences(test_df["data"],
                                                       maxlen=150000,
                                                       dtype="float32")[:, :, np.newaxis]
if augmentation == True:
    X = augment(samples=X, sample_rate=sampling_rate)

y = train_df["covid19"]

X.shape, y.shape, X_test.shape

((3805, 150000, 1), (3805,), (5732, 150000, 1))

In [5]:
def preprocess_feature(df):
    temp = df.copy()
    temp["condition1"] = temp["respiratory_condition"] + temp["fever_or_muscle_pain"]
    temp["condition2"] = temp["respiratory_condition"] * temp["fever_or_muscle_pain"]
    temp = temp.drop(["id", "age", "gender", "respiratory_condition", "fever_or_muscle_pain"], axis=1)
    return temp

train_df = preprocess_feature(train_df)
test_df = preprocess_feature(test_df)

X_test_tab = test_df[["condition1", "condition2"]].values

X_test_tab.shape

(5732, 2)

## Training

In [6]:
def scratch_1dcnn():
    
    inp = tf.keras.Input(shape=(X.shape[1], X.shape[2]))
    tab = tf.keras.Input(shape=(X_test_tab.shape[1],))
    
    x = layers.Conv1D(16, 9, 4, padding="same")(inp)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(16, 9, 4, padding="same")(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv1D(32, 8, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(32, 8, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv1D(64, 7, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(64, 7, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv1D(128, 6, 2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(128, 6, 2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv1D(256, 5, 2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(256, 5, 2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv1D(512, 4, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(512, 4, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv1D(1024, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(1024, 3, 1, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Concatenate()([x, tab])
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(16, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    
    oup = layers.Dense(1, activation="sigmoid")(x)
    
    model = tf.keras.Model(inputs=[inp, tab], outputs=oup)
    
    return model

model = scratch_1dcnn()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150000, 1)]  0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 37500, 16)    160         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 37500, 16)   64          ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 37500, 16)    0           ['batch_normalization[0][0]']

In [7]:
validations = []
thresholds = []
predictions = []

idx=0

skf = StratifiedKFold(n_splits=cv)
for train_index, val_index in tqdm(skf.split(X, y)):
    
    idx+=1

    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    X_train_tab = train_df[["condition1", "condition2"]].values[train_index]
    X_val_tab = train_df[["condition1", "condition2"]].values[val_index]
    
    train_ds = (
        tf.data.Dataset.from_tensor_slices(((X_train, X_train_tab), y_train))
        .shuffle(len(X_train))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    val_ds = (
        tf.data.Dataset.from_tensor_slices(((X_val, X_val_tab), y_val))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    model = scratch_1dcnn()

    lr = tf.keras.optimizers.schedules.CosineDecay(learning_rate, decay_steps=1000)
    if optimizer == "adam":
        optim = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "sgd":
        optim = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
        
    if loss == "bc":
        label_smoothing=0
        loss_function = tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing)
    elif loss == "fl":
        loss_function = tfa.losses.SigmoidFocalCrossEntropy()

    model.compile(
        optimizer=optim,
        loss=loss_function,
    )
    
    checkpoint_filepath=f"load_model/{parser.description}_{idx}"

    checkpoint_callback = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
        ),
        tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True
        )
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[checkpoint_callback, WandbCallback()],
    )

    max_f1 = 0
    threshold = 0
    for temp_threshold in np.linspace(0.05, 0.95, 19):
        temp_f1 = f1_score(y_val, np.where(model.predict(val_ds)>temp_threshold, 1, 0), average="macro")
        if temp_f1 > max_f1:
            max_f1 = temp_f1
            threshold = temp_threshold
    
    print(f"idx:{idx}, f1:{max_f1}, threshold:{threshold}")
    
    validations.append(max_f1)
    thresholds.append(threshold)
    predictions.append(model.predict([X_test, X_test_tab]))

val_f1 = np.mean(validations, axis=0)

print("validation_macro-f1: ", val_f1)
wandb.log({'validation_macro-f1': val_f1})

0it [00:00, ?it/s]

Epoch 1/100
107/107 [==============================] - 33s 100ms/step - loss: 0.0355 - val_loss: 0.0421 - _timestamp: 1655823059.0000 - _runtime: 66.0000
Epoch 2/100
107/107 [==============================] - 10s 91ms/step - loss: 0.0288 - val_loss: 0.0294 - _timestamp: 1655823071.0000 - _runtime: 78.0000
Epoch 3/100
107/107 [==============================] - 9s 83ms/step - loss: 0.0263 - val_loss: 0.0335 - _timestamp: 1655823082.0000 - _runtime: 89.0000
Epoch 4/100
107/107 [==============================] - 9s 83ms/step - loss: 0.0222 - val_loss: 0.0343 - _timestamp: 1655823092.0000 - _runtime: 99.0000
Epoch 5/100
107/107 [==============================] - 9s 80ms/step - loss: 0.0167 - val_loss: 0.0334 - _timestamp: 1655823102.0000 - _runtime: 109.0000
Epoch 6/100
107/107 [==============================] - 9s 80ms/step - loss: 0.0119 - val_loss: 0.0477 - _timestamp: 1655823112.0000 - _runtime: 119.0000
Epoch 7/100
107/107 [==============================] - 9s 82ms/step - loss: 0.0085 

1it [02:54, 174.38s/it]

Epoch 1/100
107/107 [==============================] - 14s 90ms/step - loss: 0.0328 - val_loss: 0.0388 - _timestamp: 1655823213.0000 - _runtime: 220.0000
Epoch 2/100
107/107 [==============================] - 9s 83ms/step - loss: 0.0282 - val_loss: 0.0323 - _timestamp: 1655823224.0000 - _runtime: 231.0000
Epoch 3/100
107/107 [==============================] - 10s 98ms/step - loss: 0.0253 - val_loss: 0.0294 - _timestamp: 1655823235.0000 - _runtime: 242.0000
Epoch 4/100
107/107 [==============================] - 9s 86ms/step - loss: 0.0196 - val_loss: 0.0287 - _timestamp: 1655823246.0000 - _runtime: 253.0000
Epoch 5/100
107/107 [==============================] - 9s 80ms/step - loss: 0.0146 - val_loss: 0.0301 - _timestamp: 1655823256.0000 - _runtime: 263.0000
Epoch 6/100
107/107 [==============================] - 9s 80ms/step - loss: 0.0098 - val_loss: 0.0385 - _timestamp: 1655823266.0000 - _runtime: 273.0000
Epoch 7/100
107/107 [==============================] - 8s 79ms/step - loss: 0.00

2it [05:46, 173.09s/it]

...

Epoch 1/100
108/108 [==============================] - 13s 89ms/step - loss: 0.0370 - val_loss: 0.0368 - _timestamp: 1655824388.0000 - _runtime: 1395.0000
Epoch 2/100
108/108 [==============================] - 9s 84ms/step - loss: 0.0288 - val_loss: 0.0311 - _timestamp: 1655824399.0000 - _runtime: 1406.0000
Epoch 3/100
108/108 [==============================] - 9s 86ms/step - loss: 0.0252 - val_loss: 0.0298 - _timestamp: 1655824409.0000 - _runtime: 1416.0000
Epoch 4/100
108/108 [==============================] - 9s 85ms/step - loss: 0.0206 - val_loss: 0.0270 - _timestamp: 1655824420.0000 - _runtime: 1427.0000
Epoch 5/100
108/108 [==============================] - 9s 80ms/step - loss: 0.0162 - val_loss: 0.0329 - _timestamp: 1655824430.0000 - _runtime: 1437.0000
Epoch 6/100
108/108 [==============================] - 9s 80ms/step - loss: 0.0119 - val_loss: 0.0385 - _timestamp: 1655824440.0000 - _runtime: 1447.0000
Epoch 7/100
108/108 [==============================] - 9s 81ms/step - loss:

9it [25:21, 166.48s/it]

Epoch 1/100
108/108 [==============================] - 13s 87ms/step - loss: 0.0362 - val_loss: 0.0403 - _timestamp: 1655824558.0000 - _runtime: 1565.0000
Epoch 2/100
108/108 [==============================] - 9s 83ms/step - loss: 0.0279 - val_loss: 0.0288 - _timestamp: 1655824569.0000 - _runtime: 1576.0000
Epoch 3/100
108/108 [==============================] - 8s 78ms/step - loss: 0.0254 - val_loss: 0.0289 - _timestamp: 1655824579.0000 - _runtime: 1586.0000
Epoch 4/100
108/108 [==============================] - 9s 81ms/step - loss: 0.0201 - val_loss: 0.0270 - _timestamp: 1655824589.0000 - _runtime: 1596.0000
Epoch 5/100
108/108 [==============================] - 9s 79ms/step - loss: 0.0158 - val_loss: 0.0278 - _timestamp: 1655824599.0000 - _runtime: 1606.0000
Epoch 6/100
108/108 [==============================] - 9s 79ms/step - loss: 0.0105 - val_loss: 0.0368 - _timestamp: 1655824609.0000 - _runtime: 1616.0000
Epoch 7/100
108/108 [==============================] - 9s 79ms/step - loss:

10it [28:12, 169.25s/it]

validation_macro-f1:  0.5349152473904899


## Inference

In [8]:
threshold = np.mean(thresholds)
test_df["covid19"] = np.where(np.mean(predictions, axis=0)>threshold, 1, 0)

submission = pd.read_csv('data/sample_submission.csv')
submission['covid19'] = test_df['covid19']
submission.to_csv('submission.csv', index=False)

test_df['covid19'].value_counts(normalize=True)

0    0.964061
1    0.035939
Name: covid19, dtype: float64